In [8]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from qiskit import IBMQ, Aer
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_ibm_provider import IBMProvider


# Set up the provider
provider = IBMProvider(instance='ibm-q-asu/main/pi-deluca')

# Load the Iris dataset
iris = datasets.load_iris()

# Preprocess the data
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target labels
encoder = OneHotEncoder(sparse=False)
y_encoded = encoder.fit_transform(y)

# Scale the data
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Define the feature map and ansatz
feature_map = ZZFeatureMap(feature_dimension=X_train.shape[1], reps=1)
ansatz = RealAmplitudes(num_qubits=X_train.shape[1], reps=3)

# Define the quantum instance
backend = provider.get_backend('simulator_statevector')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=42, seed_transpiler=42)

# Create a VQC classifier
vqc = VQC(optimizer=COBYLA(maxiter=40), feature_map=feature_map, ansatz=ansatz, quantum_instance=quantum_instance)

# Train the classifier
vqc.fit(X_train, y_train)

# Test the classifier
y_pred = vqc.predict(X_test)

# Decode the one-hot encoded predictions
y_pred_decoded = np.argmax(y_pred, axis=1)

# Decode the one-hot encoded target labels
y_test_decoded = np.argmax(y_test, axis=1)

# Calculate the accuracy
accuracy = np.sum(y_pred_decoded == y_test_decoded) / len(y_test_decoded)
print(f"Accuracy: {accuracy * 100:.2f}%")


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Accuracy: 40.00%
